# [GoogleNet Implementation](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/43022.pdf)

- 2014 ILSVRC 1nd place 
- Inception Module

![alt text](GoogleNet.jpeg)

In [1]:
try:
    shutil.rmtree('runs/')
except:
    pass

## 1. Settings
### 1) Import libs

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data
import numpy as np
import torchvision.transforms as transforms
import torchvision.datasets as vdatasets
import torchvision.utils as vutils
import random
import os, pickle
from tensorboardX import SummaryWriter
torch.manual_seed(1)

# DATA_PATH = os.environ['/data/']
USE_CUDA = torch.cuda.is_available()

import matplotlib.pyplot as plt
%matplotlib inline

print(USE_CUDA)

False


### 2) Hyperameters

In [3]:
batch_size= 1
learning_rate = 0.0002
epoch = 100

## 2. Data Loader

In [4]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

# train
img_dir = "./data"
img_data = vdatasets.ImageFolder(img_dir, transforms.Compose([
            transforms.RandomSizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize
            ]))

img_batch = data.DataLoader(img_data, batch_size=batch_size,
                            shuffle=True, num_workers=2)

## 3. Model

## 1) Inception-A

![alt text](Inception-A.png)

In [5]:
class Inception(nn.Module):
    
    def __init__(self,in_ch,out_ch1,mid_ch13,out_ch13,mid_ch15,out_ch15,out_ch_pool_conv,auxiliary=False):
        super(Inception,self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_ch,out_ch1,kernel_size=1,stride=1),
            nn.ReLU())
        self.conv13 = nn.Sequential(
            nn.Conv2d(in_ch,mid_ch13,kernel_size=1,stride=1),
            nn.ReLU(),
            nn.Conv2d(mid_ch13,out_ch13,kernel_size=3,stride=1,padding=1),
            nn.ReLU())
        
        self.conv15 = nn.Sequential(
            nn.Conv2d(in_ch,mid_ch15,kernel_size=1,stride=1),
            nn.ReLU(),
            nn.Conv2d(mid_ch15,out_ch15,kernel_size=5,stride=1,padding=2),
            nn.ReLU())
        
        self.pool_conv1 = nn.Sequential(
            nn.MaxPool2d(3,stride=1,padding=1),
            nn.Conv2d(in_ch,out_ch_pool_conv,kernel_size=1,stride=1),
            nn.ReLU())
        
        self.auxiliary = auxiliary
        
        if auxiliary:
            self.auxiliary_layer = nn.Sequential(
                nn.AvgPool2d(5,3),
                nn.Conv2d(in_ch,128,1),
                nn.ReLU())
        
    def forward(self,inputs,train=False):
        conv1_out = self.conv1(inputs)
        conv13_out = self.conv13(inputs)
        conv15_out = self.conv15(inputs)
        pool_conv_out = self.pool_conv1(inputs)
        outputs = torch.cat([conv1_out,conv13_out,conv15_out,pool_conv_out],1) # depth-wise concat
        
        if self.auxiliary:
            if train:
                outputs2 = self.auxiliary_layer(inputs)
            else:
                outputs2 = None
            return outputs, outputs2
        else:
            return outputs

### tip) Check Inception-A

In [6]:
writer = SummaryWriter(comment='-basic-inception')
sample_images = torch.randn(2, 64, 56, 56) # batche_size, input_channel, width, height
inception = Inception(64, 32, 32, 64, 16, 32, 64)
# output = inception(Variable(sample_images))

torch.onnx.export(inception, Variable(sample_images), 'inception.proto', verbose=True)
writer.add_graph_onnx('inception.proto')
writer.close()

graph(%1 : Float(2, 64, 56, 56)
      %2 : Float(32, 64, 1, 1)
      %3 : Float(32)
      %4 : Float(32, 64, 1, 1)
      %5 : Float(32)
      %6 : Float(64, 32, 3, 3)
      %7 : Float(64)
      %8 : Float(16, 64, 1, 1)
      %9 : Float(16)
      %10 : Float(32, 16, 5, 5)
      %11 : Float(32)
      %12 : Float(64, 64, 1, 1)
      %13 : Float(64)) {
  %15 : UNKNOWN_TYPE = Conv[kernel_shape=[1, 1], strides=[1, 1], pads=[0, 0, 0, 0], dilations=[1, 1], group=1](%1, %2), uses = [[%16.i0]], scope: Inception/Sequential[conv1]/Conv2d[0];
  %16 : Float(2, 32, 56, 56) = Add[broadcast=1, axis=1](%15, %3), uses = [%17.i0], scope: Inception/Sequential[conv1]/Conv2d[0];
  %17 : Float(2, 32, 56, 56) = Relu(%16), uses = [%39.i0], scope: Inception/Sequential[conv1]/ReLU[1];
  %19 : UNKNOWN_TYPE = Conv[kernel_shape=[1, 1], strides=[1, 1], pads=[0, 0, 0, 0], dilations=[1, 1], group=1](%1, %4), uses = [[%20.i0]], scope: Inception/Sequential[conv13]/Conv2d[0];
  %20 : Float(2, 32, 56, 56) = Add[broadcast=1

```cell
source activate env

cd /path/

tensorboard --logdir runs
```

## 2) GoogLeNet

In [7]:
class GoogLeNet(nn.Module):
    
    def __init__(self,num_output=1000):
        super(GoogLeNet,self).__init__()
        
        self.stem_layer = nn.Sequential(
                                                        nn.Conv2d(3,64,7,2,3),
                                                        nn.ReLU(),
                                                        nn.MaxPool2d(3,2,1),
                                                        nn.Conv2d(64,64,1),
                                                        nn.ReLU(),
                                                        nn.Conv2d(64,192,3,1,1),
                                                        nn.ReLU(),
                                                        nn.MaxPool2d(3,2,1)
                                                        )
        
        #in_ch,out_ch_1,mid_ch_13,out_ch_13,mid_ch_15,out_ch_15,out_ch_pool_conv
        self.inception_layer1 = nn.Sequential(
                                                                Inception(192,64,96,128,16,32,32),
                                                                Inception(256,128,128,192,32,96,64),
                                                                nn.MaxPool2d(3,2,1)
                                                               )
        
        self.inception_layer2 = nn.Sequential(
                                                                Inception(480,192,96,208,16,48,64),
                                                                Inception(512,160,112,224,24,64,64),
                                                                Inception(512,128,128,256,24,64,64),
                                                                Inception(512,112,144,288,32,64,64),
                                                                Inception(528,256,160,320,32,128,128),
                                                                nn.MaxPool2d(3,2,1)
                                                               )
        
        #self.inception_layer3 = Inception(528,256,160,320,32,128,128,True) # auxiliary classifier
        #self.auxiliary_layer = nn.Linear(128*4*4,num_output)
        
        self.inception_layer3 = nn.Sequential(
                                                                #nn.MaxPool2d(3,2,1),
                                                                Inception(832,256,160,320,32,128,128),
                                                                Inception(832,384,192,384,48,128,128),
                                                                nn.AvgPool2d(7,1)
                                                               )
        
        self.dropout = nn.Dropout2d(0.4)
        self.output_layer = nn.Linear(1024,num_output)
        
    def forward(self,inputs,train=False):
        outputs = self.stem_layer(inputs)
        outputs = self.inception_layer1(outputs)
        outputs = self.inception_layer2(outputs)
        #outputs,outputs2 = self.inception_layer3(outputs)
        #if train:
            # B,128,4,4 => B,128*4*4
        #    outputs2 = self.auxiliary_layer(outputs2.view(inputs.size(0),-1))
        outputs = self.inception_layer3(outputs)
        outputs = self.dropout(outputs)
        outputs = outputs.view(outputs.size(0),-1) # 동일 : outputs = outputs.view(batch_size,-1)
        outputs = self.output_layer(outputs)
        
        #if train:
        #   return outputs, outputs2
        return outputs

### tip) check GoogLeNet

In [8]:
model = GoogLeNet(2)
writer = SummaryWriter(comment="-googlenet")

In [9]:
sample_x = torch.randn(batch_size,3,224,224)
sample_y = torch.randperm(batch_size)

In [10]:
preds = model(Variable(sample_x))
torch.onnx.export(model, Variable(sample_x), 'Googlenet.proto', verbose=True)
writer.add_graph_onnx('Googlenet.proto')
writer.close()

graph(%1 : Float(1, 3, 224, 224)
      %2 : Float(64, 3, 7, 7)
      %3 : Float(64)
      %4 : Float(64, 64, 1, 1)
      %5 : Float(64)
      %6 : Float(192, 64, 3, 3)
      %7 : Float(192)
      %8 : Float(64, 192, 1, 1)
      %9 : Float(64)
      %10 : Float(96, 192, 1, 1)
      %11 : Float(96)
      %12 : Float(128, 96, 3, 3)
      %13 : Float(128)
      %14 : Float(16, 192, 1, 1)
      %15 : Float(16)
      %16 : Float(32, 16, 5, 5)
      %17 : Float(32)
      %18 : Float(32, 192, 1, 1)
      %19 : Float(32)
      %20 : Float(128, 256, 1, 1)
      %21 : Float(128)
      %22 : Float(128, 256, 1, 1)
      %23 : Float(128)
      %24 : Float(192, 128, 3, 3)
      %25 : Float(192)
      %26 : Float(32, 256, 1, 1)
      %27 : Float(32)
      %28 : Float(96, 32, 5, 5)
      %29 : Float(96)
      %30 : Float(64, 256, 1, 1)
      %31 : Float(64)
      %32 : Float(192, 480, 1, 1)
      %33 : Float(192)
      %34 : Float(96, 480, 1, 1)
      %35 : Float(96)
      %36 : Float(208, 96, 3, 3)
  

## 4. Optimizer & Loss

In [11]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## 5. Train

In [ ]:
# model.train()

In [12]:
for i in range(epoch):
    for img,label in img_batch:
        img = Variable(img)#.cuda()
        label = Variable(label)#.cuda()

        optimizer.zero_grad()
        output = model(img)
        
        #print(output.size())
        loss = loss_func(output,label)
        loss.backward()
        optimizer.step()

    if i % 10 ==0:
        print(loss)

Variable containing:
 0.7446
[torch.FloatTensor of size 1]

Variable containing:
 0.6864
[torch.FloatTensor of size 1]

Variable containing:
 0.7023
[torch.FloatTensor of size 1]

Variable containing:
 0.7096
[torch.FloatTensor of size 1]

Variable containing:
 0.6932
[torch.FloatTensor of size 1]

Variable containing:
 0.7054
[torch.FloatTensor of size 1]

Variable containing:
 0.6839
[torch.FloatTensor of size 1]

Variable containing:
 0.6715
[torch.FloatTensor of size 1]

Variable containing:
 0.6886
[torch.FloatTensor of size 1]

Variable containing:
 0.7059
[torch.FloatTensor of size 1]

